In [4]:
# Create a reference to the CSV and import it into a Pandas DataFrame
csv_path = Path("../Resources/Bedbug_Reporting.csv")
pd.read_csv(csv_path)

Building ID  Registration ID        Borough House Number  \
0             14135           117174      MANHATTAN            5   
1            806924           206329          BRONX         2400   
2             14951           106899      MANHATTAN          348   
3            661202           407317         QUEENS         1714   
4            425643           423224         QUEENS       142-36   
...             ...              ...            ...          ...   
163243       333526           309902       BROOKLYN           20   
163244       308761           325711       BROOKLYN          793   
163245        22918           139527      MANHATTAN          105   
163246       896705           143111      MANHATTAN           80   
163247       900236            16221  STATEN ISLAND           50   

                Street Name  Postcode  # of Dwelling Units  \
0            EAST 51 STREET   10022.0                 14.0   
1           SEDGWICK AVENUE   10468.0                128.0   
2            EAST 62 STREET   10065.0                 22.0   
3              GROVE STREET   11385.0                  3.0   
4                 38 AVENUE   11354.0                 18.0   
...                     ...       ...                  ...   
163243        MAUJER STREET   11206.0                 16.0   
163244          HART STREET   11237.0                  6.0   
163245        HUDSON STREET   10013.0                 41.0   
163246  RIVERSIDE BOULEVARD   10069.0                270.0   
163247         RUGBY AVENUE   10301.0                  2.0   

        Infested Dwelling Unit Count  Eradicated Unit Count  \
0                                0.0                    0.0   
1                                0.0                    0.0   
2                                0.0                    0.0   
3                                0.0                    0.0   
4                                0.0                    0.0   
...                              ...                    ...   
163243                           0.0                    0.0   
163244                           0.0                    0.0   
163245                           0.0                    0.0   
163246                           0.0                    0.0   
163247                           0.0                    0.0   

        Re-infested  Dwelling Unit Count  ... Filing Period Start Date  \
0                                    0.0  ...               11/01/2019   
1                                    0.0  ...               11/01/2019   
2                                    0.0  ...               11/01/2019   
3                                    0.0  ...               11/01/2019   
4                                    0.0  ...               11/01/2019   
...                                  ...  ...                      ...   
163243                               0.0  ...               11/01/2019   
163244                               0.0  ...               11/01/2018   
163245                               0.0  ...               11/01/2019   
163246                               0.0  ...               11/01/2019   
163247                               0.0  ...               11/01/2019   

       Filling Period End Date   Latitude  Longitude  Community Board  \
0                   10/31/2020  40.758998 -73.976324              5.0   
1                   10/31/2020  40.864184 -73.908991              7.0   
2                   10/31/2020  40.761862 -73.961509              8.0   
3                   10/31/2020  40.703275 -73.910772              5.0   
4                   10/31/2020  40.761839 -73.826239              7.0   
...                        ...        ...        ...              ...   
163243              10/31/2020  40.710303 -73.950050              1.0   
163244              10/31/2019  40.701077 -73.923592              4.0   
163245              10/31/2020  40.719396 -74.008690              1.0   
163246              10/31/2020  40.775485 -73.990230              7.0 

In [8]:
# Columns we're interested in: 'Building ID', 'Borough', 'Postcode', '# of Dwelling Units',
#       'Infested Dwelling Unit Count', 'Eradicated Unit Count',
#       'Re-infested Dwelling Unit Count', 'Filing Date', 'Latitude', 'Longitude'
bugs_df = bugs_df[['Building ID', 'Borough', 'Postcode', '# of Dwelling Units',
       'Infested Dwelling Unit Count', 'Eradicated Unit Count',
       'Re-infested Dwelling Unit Count', 'Filing Date',
       'Latitude', 'Longitude']]
bugs_df.head()

In [ ]:
# Extract the year from the date
bugs_df["Year"] = bugs_df["Filing Date"].dt.year
bugs_df.head()

In [ ]:
# Filter to only buildings with infested units greater than 0
bug_infestations = bugs_df.loc[(bugs_df["Infested Dwelling Unit Count"]>0),:]
bug_infestations.head()

In [ ]:
# Change postcode to int
bug_infestations["Postcode"] = bug_infestations["Postcode"].astype("int64")
bug_infestations.head()

In [ ]:
# Create a column for percentage of units infested
bug_infestations["Percent Units Infested"] = bug_infestations["Infested Dwelling Unit Count"] /
                                                bug_infestations["# of Dwelling Units"] * 100
bug_infestations.head()

In [ ]:
# Finding the average percentage of infested units
average_infested_units = bug_infestations["Percent Units Infested"].mean()
average_infested_units

In [ ]:
# Grouping the DataFrame by "Year"
year_group = bug_infestations.groupby("Year")

# Count how many buildings were infested in each borough and create DataFrame
year_borough_df = pd.DataFrame(year_group["Borough"].value_counts())
year_borough_df.head(10)

In [ ]:
# Rename the "Borough" column to "Total Building Infestations"
year_borough_df = year_borough_df.rename(
    columns={"Borough": "Total Building Infestations"})
year_borough_df.head()

In [ ]:
# Create a DataFrame that shows the total infested and re-infested dwelling unit count by year and borough
year_borough_group = bug_infestations.groupby(["Year", "Borough"])
unit_infestations_by_year_borough = pd.DataFrame(year_borough_group[["Infested Dwelling Unit Count",
                                                                   "Re-infested Dwelling Unit Count"]].sum())
unit_infestations_by_year_borough

In [ ]:
# Find the total unit infestations and re-infestations by year
total_unit_infestations_each_year = pd.DataFrame(year_group["Infested Dwelling Unit Count", 
                                                             "Re-infested Dwelling Unit Count"].sum())
total_unit_infestations_each_year = total_unit_infestations_each_year\
            .rename(columns={"Infested Dwelling Unit Count": "Total Infested Dwelling Units in Year",
                            "Re-infested Dwelling Unit Count": "Total Re-infested Dwelling Units in Year"})
total_unit_infestations_each_year

In [ ]:
# Merge unit_infestations_by_year_borough and join the "Total Infested Dwelling Units in Year"
# into the year_borough_df DataFrame
merged_df = year_borough_df.merge(unit_infestations_by_year_borough).join(total_unit_infestations_each_year,
                                                               on="Year")
merged_df.head()